# Multivariante del método K-Nearest Neighbors

Hasta ahora exploramos como usar un modelo de ML, el KNN simple que uso solo una caracteristica para las predicciones. Si bien esto ayudo a familiarizarse con los conceptos basicos del ML, esta claro que usar una sola caracteristica no refleja la realidad.<br>

Hay 2 formas en las que podemos retocar este modelo para tratar de mejorar la precision (mejorar el RMSE):<br>
- Incrementar el numero de atributos (**caracteristicas**) que el modelo usa para calcular similitudes cuando ranque los vecinos mas cercanos.
- Incrementar **k**, el numero de vecinos cercanos que el modelo usa.

### Eliminacion de caracteristicas no utiles y _**Normalizacion**_ de los datos.

Al seleccionar mas atributos, tenemos que tener cuidado con las columnas que no funcionan bien con la ecuacion de distancia:
- Valores no numericos (ejemplo, ciudad o estado)
    - La ecuacion de distancia euclidiana espera valores numericos.
- Valores perdidos
    - La ecuacion de la distancia euclideana espera un valor por cada observacion y atributo.
- Valores no ordinales (por ejemplo, latitud o longitud)
    - La clasifiacion por distancia euclideana no tiene sentido si todos los valores no son ordinales. 

In [1]:
import pandas as pd
import numpy as np

# Leyendo datos y acomodandolos aleatoreamente
dc_listings = pd.read_csv('dc_airbnb.csv')
dc_listings = dc_listings.loc[ np.random.permutation(len(dc_listings)) ]

# Convirtiendo la columna price a float (limpiando datos)
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')

dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state
1170,100%,96%,2,2,Private room,1.0,1.0,1.0,89.0,NaN,NaN,2,1125,6,38.913256,-77.044239,Washington,20009,DC
1453,100%,100%,1,2,Entire home/apt,2.0,2.0,2.0,250.0,NaN,NaN,3,10,1,38.871283,-77.018715,Washington,20024,DC
2985,NaN,NaN,1,6,Entire home/apt,3.0,2.0,3.0,600.0,NaN,NaN,1,1125,0,38.923560,-77.024338,Washington,20001,DC
3397,NaN,NaN,1,1,Private room,1.0,1.0,1.0,96.0,NaN,NaN,4,7,0,38.928433,-77.057239,Washington,20008,DC
1102,92%,99%,23,2,Private room,1.0,1.0,1.0,120.0,NaN,NaN,1,1125,4,38.915944,-77.038662,Washington,20009,DC


In [2]:
# Mostrando informacion general de los datos. 
# Como: columnas, cuantos valores no nulos contiene cada columna y el tipo de dato
print(dc_listings.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3723 entries, 1170 to 153
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   host_response_rate    3289 non-null   object 
 1   host_acceptance_rate  3109 non-null   object 
 2   host_listings_count   3723 non-null   int64  
 3   accommodates          3723 non-null   int64  
 4   room_type             3723 non-null   object 
 5   bedrooms              3702 non-null   float64
 6   bathrooms             3696 non-null   float64
 7   beds                  3712 non-null   float64
 8   price                 3723 non-null   float64
 9   cleaning_fee          2335 non-null   object 
 10  security_deposit      1426 non-null   object 
 11  minimum_nights        3723 non-null   int64  
 12  maximum_nights        3723 non-null   int64  
 13  number_of_reviews     3723 non-null   int64  
 14  latitude              3723 non-null   float64
 15  longitude             37

Como podemos observar: 
- Hay columnas que contienen valores _**NO númericos**_:
    - _room_type_
    - _city_
    - _state_
- Y columnas con valores númericos pero _**NO ordinales**_:
    - _latitude_
    - _longitude_
    - _zipcode_


A su vez tambien evitaremos cualquier columna que no describa directamente el espacio vital:
- _host_response_rate_
- _host_acceptance_rate_
- _host_listings_count_


In [4]:
# Eliminando columnas no utiles.
drop_columns = [ 'room_type', 'city', 'state', 'latitude', 'longitude', 'zipcode', 'host_response_rate', 'host_acceptance_rate', 'host_listings_count' ]
dc_listings = dc_listings.drop(drop_columns, axis=1)
dc_listings.isnull().sum()

accommodates            0
bedrooms               21
bathrooms              27
beds                   11
price                   0
cleaning_fee         1388
security_deposit     2297
minimum_nights          0
maximum_nights          0
number_of_reviews       0
dtype: int64

De las columnas restantes, hay algunas que tienen pocos valores nulos:
- _bedrooms_
- _bathrooms_
- _beds_

De las cuales podemos eliminar esas filas sin perder mucha informacion.

Pero tambien hay 2 columnas con un gran numero de valores perdidos:
- _cleaning_fee_
- _security_deposit_

En este caso no podemos eliminar simplemente las filas, ya que perderiamos mucha informacion. Por ello, vamos a eliminar <br>
por completo esas dos columas.

In [5]:
# Eliminando columnas con un gran numero de valores perdidos
dc_listings = dc_listings.drop([ 'cleaning_fee', 'security_deposit' ], axis=1)

# Eliminando filas con valores nulos
dc_listings = dc_listings.dropna(axis=0)

dc_listings.isnull().sum()

accommodates         0
bedrooms             0
bathrooms            0
beds                 0
price                0
minimum_nights       0
maximum_nights       0
number_of_reviews    0
dtype: int64